In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
from pathlib import Path
drive_base = '/content/drive/MyDrive/Colab Notebooks/image_retrieval/dataset/fashion-iq'
colab_base = '/content/fashion-iq'

# Create destination root in Colab internal storage
os.makedirs(colab_base, exist_ok=True)

# Define subdirectories
subdirs = ['image_splits', 'captions']

# Copy 'image_splits' and 'captions' directories
for subdir in subdirs:
    src = os.path.join(drive_base, subdir)
    dst = os.path.join(colab_base, subdir)
    if os.path.exists(dst):
        shutil.rmtree(dst)  # Remove existing to avoid duplication/conflict
    shutil.copytree(src, dst)
    print(f"✅ Copied {subdir} to {dst}")

# Handle images.zip
zip_path = os.path.join(drive_base, 'images/images.zip')
extract_dir = os.path.join(colab_base, 'images')

# Make sure destination for images exists
os.makedirs(extract_dir, exist_ok=True)

# Unzip images.zip into /content/fashion-iq/images/
!unzip -q "$zip_path" -d "$extract_dir"

print("✅ Unzipped images.zip into Colab storage.")

✅ Copied image_splits to /content/fashion-iq/image_splits
✅ Copied captions to /content/fashion-iq/captions
replace /content/fashion-iq/images/images/245600258X.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ✅ Unzipped images.zip into Colab storage.


In [ ]:
import os
import shutil
from pathlib import Path
drive_base = '/content/drive/MyDrive/Colab Notebooks/image_retrieval/dataset/fashion-iq'
colab_base = '/content/fashion-iq'
base_path =  Path("/content/fashion-iq")

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/image_retrieval')
import importlib
import data_utils  # initial import
from data_utils import targetpad_transform,_convert_image_to_rgb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Copied image_splits to /content/fashion-iq/image_splits
✅ Copied captions to /content/fashion-iq/captions


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-io4dn95y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-io4dn95y
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import clip
import torch
from PIL import Image


In [ ]:
transform = targetpad_transform(target_ratio=1.0, dim=224)


In [ ]:
prompts = [
    "a clothing item",
    "a fashion dress",
    "a shirt or top",
    "a brand logo",
    "an image with only text",
    "an empty product image",
]

In [ ]:
model, preprocess_clip = clip.load("ViT-B/32", device=device)
text_tokens = clip.tokenize(prompts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)  # Normalize

In [ ]:
from pathlib import Path
from tqdm import tqdm

In [ ]:
image_folder = base_path / 'images' / 'images'
output_folder = Path("/content/fashion-iq-cleaned")
output_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
unrelated_images = []
for img_path in tqdm(list(image_folder.glob("*.png"))):
    try:
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            img_features = model.encode_image(img_tensor)
            img_features /= img_features.norm(dim=-1, keepdim=True)

            similarity = (100.0 * img_features @ text_features.T).softmax(dim=-1)
            best_prompt_idx = similarity.argmax().item()
            best_prompt = prompts[best_prompt_idx]

            if best_prompt in ["a brand logo", "an image with only text", "an empty product image"]:
                unrelated_images.append(str(img_path))
            else:
                shutil.copy(img_path, output_folder / img_path.name)
    except Exception as e:
        print(f"⚠️ Error reading {img_path.name}: {e}")

100%|██████████| 74381/74381 [17:29<00:00, 70.88it/s]


In [ ]:
# Define paths
cleaned_colab_path = "/content/fashion-iq-cleaned"
drive_destination = "/content/drive/MyDrive/Colab Notebooks/image_retrieval/dataset/fashion-iq-cleaned"

# Remove if already exists in Drive to avoid duplication (optional)
import shutil
import os

if os.path.exists(drive_destination):
    shutil.rmtree(drive_destination)

# Copy the cleaned folder to your Drive
shutil.copytree(cleaned_colab_path, drive_destination)

print("✅ Cleaned dataset successfully saved to Google Drive!")


✅ Cleaned dataset successfully saved to Google Drive!


In [ ]:
cleaned_path = "/content/fashion-iq-cleaned"

# Count image files (e.g., .png, .jpg, .jpeg)
image_extensions = (".png", ".jpg", ".jpeg")
image_count = sum(
    1 for fname in os.listdir(cleaned_path) if fname.lower().endswith(image_extensions)
)

print(f"✅ Number of images in '{cleaned_path}': {image_count}")


✅ Number of images in '/content/fashion-iq-cleaned': 73620


In [ ]:
import os
import json
from pathlib import Path
from PIL import Image
from collections import Counter
from sklearn.cluster import KMeans
import numpy as np
import webcolors
from tqdm import tqdm

# Install webcolors if not installed
!pip install -q webcolors

# Helper functions for color processing
def extract_dominant_color(image_path, k=4):
    try:
        image = Image.open(image_path).convert('RGB')
        image = image.resize((100, 100))  # Resize for speed
        image_np = np.array(image).reshape((-1, 3))

        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(image_np)

        # Find the most common cluster
        counts = Counter(kmeans.labels_)
        dominant_cluster = counts.most_common(1)[0][0]
        dominant_color = kmeans.cluster_centers_[dominant_cluster].astype(int)
        return tuple(dominant_color)
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

from webcolors import CSS3_HEX_TO_NAMES, hex_to_rgb, rgb_to_name

def closest_color(requested_color):
    min_colors = {}
    for hex_value, name in CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = hex_to_rgb(hex_value)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def rgb_to_name(rgb_tuple):
    try:
        return rgb_to_name(rgb_tuple, spec='css3')
    except ValueError:
        return closest_color(rgb_tuple)


# === Paths and setup ===
base_path = Path("/content/fashion-iq")
cleaned_images_dir = Path("/content/fashion-iq-cleaned")
captions_dir = base_path / "captions"
output_dir = base_path / "captions"/"captions_with_colors2"
output_dir.mkdir(exist_ok=True)

dress_types = ['dress', 'shirt', 'toptee']
splits = ['train', 'val', 'test']  # or just ['train'] if needed

# === Loop through all caption files ===
for dress_type in dress_types:
    for split in splits:
        caption_file = captions_dir / f"cap.{dress_type}.{split}.json"
        if not caption_file.exists():
            print(f"❌ Skipping missing file: {caption_file}")
            continue

        print(f"🎨 Processing {caption_file}...")
        with open(caption_file, 'r') as f:
            triplets = json.load(f)

        for triplet in tqdm(triplets):
            candidate_path = cleaned_images_dir / f"{triplet['candidate']}.png"
            target_path = cleaned_images_dir / f"{triplet['target']}.png"

            candidate_color = extract_dominant_color(candidate_path)
            target_color = extract_dominant_color(target_path)

            triplet['candidate_color_rgb'] = candidate_color
            triplet['candidate_color_name'] = rgb_to_name(candidate_color) if candidate_color else None
            triplet['target_color_rgb'] = target_color
            triplet['target_color_name'] = rgb_to_name(target_color) if target_color else None

        # Save updated JSON
        out_file = output_dir / f"cap.{dress_type}.{split}.json"
        with open(out_file, 'w') as f:
            json.dump(triplets, f, indent=2)
        print(f"✅ Saved updated triplets to {out_file}")


ImportError: cannot import name 'CSS3_HEX_TO_NAMES' from 'webcolors' (/usr/local/lib/python3.11/dist-packages/webcolors/__init__.py)

In [ ]:
import os
import json
from pathlib import Path
from PIL import Image
from collections import Counter
from sklearn.cluster import KMeans
import numpy as np
import webcolors
from tqdm import tqdm

# Install webcolors if not installed
# !pip install -q webcolors

# Helper functions for color processing
def extract_dominant_color(image_path, k=4):
    try:
        image = Image.open(image_path).convert('RGB')
        image = image.resize((100, 100))  # Resize for speed
        image_np = np.array(image).reshape((-1, 3))

        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(image_np)

        # Find the most common cluster
        counts = Counter(kmeans.labels_)
        dominant_cluster = counts.most_common(1)[0][0]
        dominant_color = kmeans.cluster_centers_[dominant_cluster].astype(int)
        # Convert numpy integers to regular Python integers for JSON serialization
        return tuple(int(x) for x in dominant_color)
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

def closest_color(requested_color):
    """Find the closest CSS3 color name for an RGB tuple"""
    min_colors = {}

    # Define basic CSS3 colors manually since webcolors API varies
    css3_colors = {
        'aliceblue': '#f0f8ff',
        'antiquewhite': '#faebd7',
        'aqua': '#00ffff',
        'aquamarine': '#7fffd4',
        'azure': '#f0ffff',
        'beige': '#f5f5dc',
        'bisque': '#ffe4c4',
        'black': '#000000',
        'blanchedalmond': '#ffebcd',
        'blue': '#0000ff',
        'blueviolet': '#8a2be2',
        'brown': '#a52a2a',
        'burlywood': '#deb887',
        'cadetblue': '#5f9ea0',
        'chartreuse': '#7fff00',
        'chocolate': '#d2691e',
        'coral': '#ff7f50',
        'cornflowerblue': '#6495ed',
        'cornsilk': '#fff8dc',
        'crimson': '#dc143c',
        'cyan': '#00ffff',
        'darkblue': '#00008b',
        'darkcyan': '#008b8b',
        'darkgoldenrod': '#b8860b',
        'darkgray': '#a9a9a9',
        'darkgreen': '#006400',
        'darkkhaki': '#bdb76b',
        'darkmagenta': '#8b008b',
        'darkolivegreen': '#556b2f',
        'darkorange': '#ff8c00',
        'darkorchid': '#9932cc',
        'darkred': '#8b0000',
        'darksalmon': '#e9967a',
        'darkseagreen': '#8fbc8f',
        'darkslateblue': '#483d8b',
        'darkslategray': '#2f4f4f',
        'darkturquoise': '#00ced1',
        'darkviolet': '#9400d3',
        'deeppink': '#ff1493',
        'deepskyblue': '#00bfff',
        'dimgray': '#696969',
        'dodgerblue': '#1e90ff',
        'firebrick': '#b22222',
        'floralwhite': '#fffaf0',
        'forestgreen': '#228b22',
        'fuchsia': '#ff00ff',
        'gainsboro': '#dcdcdc',
        'ghostwhite': '#f8f8ff',
        'gold': '#ffd700',
        'goldenrod': '#daa520',
        'gray': '#808080',
        'green': '#008000',
        'greenyellow': '#adff2f',
        'honeydew': '#f0fff0',
        'hotpink': '#ff69b4',
        'indianred': '#cd5c5c',
        'indigo': '#4b0082',
        'ivory': '#fffff0',
        'khaki': '#f0e68c',
        'lavender': '#e6e6fa',
        'lavenderblush': '#fff0f5',
        'lawngreen': '#7cfc00',
        'lemonchiffon': '#fffacd',
        'lightblue': '#add8e6',
        'lightcoral': '#f08080',
        'lightcyan': '#e0ffff',
        'lightgoldenrodyellow': '#fafad2',
        'lightgray': '#d3d3d3',
        'lightgreen': '#90ee90',
        'lightpink': '#ffb6c1',
        'lightsalmon': '#ffa07a',
        'lightseagreen': '#20b2aa',
        'lightskyblue': '#87cefa',
        'lightslategray': '#778899',
        'lightsteelblue': '#b0c4de',
        'lightyellow': '#ffffe0',
        'lime': '#00ff00',
        'limegreen': '#32cd32',
        'linen': '#faf0e6',
        'magenta': '#ff00ff',
        'maroon': '#800000',
        'mediumaquamarine': '#66cdaa',
        'mediumblue': '#0000cd',
        'mediumorchid': '#ba55d3',
        'mediumpurple': '#9370db',
        'mediumseagreen': '#3cb371',
        'mediumslateblue': '#7b68ee',
        'mediumspringgreen': '#00fa9a',
        'mediumturquoise': '#48d1cc',
        'mediumvioletred': '#c71585',
        'midnightblue': '#191970',
        'mintcream': '#f5fffa',
        'mistyrose': '#ffe4e1',
        'moccasin': '#ffe4b5',
        'navajowhite': '#ffdead',
        'navy': '#000080',
        'oldlace': '#fdf5e6',
        'olive': '#808000',
        'olivedrab': '#6b8e23',
        'orange': '#ffa500',
        'orangered': '#ff4500',
        'orchid': '#da70d6',
        'palegoldenrod': '#eee8aa',
        'palegreen': '#98fb98',
        'paleturquoise': '#afeeee',
        'palevioletred': '#db7093',
        'papayawhip': '#ffefd5',
        'peachpuff': '#ffdab9',
        'peru': '#cd853f',
        'pink': '#ffc0cb',
        'plum': '#dda0dd',
        'powderblue': '#b0e0e6',
        'purple': '#800080',
        'red': '#ff0000',
        'rosybrown': '#bc8f8f',
        'royalblue': '#4169e1',
        'saddlebrown': '#8b4513',
        'salmon': '#fa8072',
        'sandybrown': '#f4a460',
        'seagreen': '#2e8b57',
        'seashell': '#fff5ee',
        'sienna': '#a0522d',
        'silver': '#c0c0c0',
        'skyblue': '#87ceeb',
        'slateblue': '#6a5acd',
        'slategray': '#708090',
        'snow': '#fffafa',
        'springgreen': '#00ff7f',
        'steelblue': '#4682b4',
        'tan': '#d2b48c',
        'teal': '#008080',
        'thistle': '#d8bfd8',
        'tomato': '#ff6347',
        'turquoise': '#40e0d0',
        'violet': '#ee82ee',
        'wheat': '#f5deb3',
        'white': '#ffffff',
        'whitesmoke': '#f5f5f5',
        'yellow': '#ffff00',
        'yellowgreen': '#9acd32'
    }

    for name, hex_value in css3_colors.items():
        # Convert hex to RGB
        r_c, g_c, b_c = webcolors.hex_to_rgb(hex_value)

        # Calculate Euclidean distance
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name

    return min_colors[min(min_colors.keys())]

def rgb_to_color_name(rgb_tuple):
    """Convert RGB tuple to the closest CSS3 color name"""
    if rgb_tuple is None:
        return None

    try:
        # Try to get exact match first
        return webcolors.rgb_to_name(rgb_tuple)
    except ValueError:
        # If no exact match, find closest color
        return closest_color(rgb_tuple)

# Helper function to find image file with different extensions
def find_image_file(base_path, image_id):
    """Try to find image file with different extensions"""
    extensions = ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG']
    for ext in extensions:
        image_path = base_path / f"{image_id}{ext}"
        if image_path.exists():
            return image_path
    return None

# === Paths and setup ===
base_path = Path("/content/fashion-iq")
cleaned_images_dir = Path("/content/fashion-iq-cleaned")
captions_dir = base_path / "captions"
output_dir = base_path / "captions" / "captions_with_colors2"
output_dir.mkdir(exist_ok=True)

dress_types = ['dress', 'shirt', 'toptee']
splits = ['train', 'val', 'test']  # or just ['train'] if needed

# === Loop through all caption files ===
for dress_type in dress_types:
    for split in splits:
        caption_file = captions_dir / f"cap.{dress_type}.{split}.json"
        if not caption_file.exists():
            print(f"❌ Skipping missing file: {caption_file}")
            continue

        print(f"🎨 Processing {caption_file}...")
        with open(caption_file, 'r') as f:
            triplets = json.load(f)

        processed_count = 0
        missing_images = []

        for triplet in tqdm(triplets):
            # Try to find candidate image with different extensions
            candidate_path = find_image_file(cleaned_images_dir, triplet['candidate'])
            target_path = find_image_file(cleaned_images_dir, triplet['target'])

            # Process candidate image
            if candidate_path:
                candidate_color = extract_dominant_color(candidate_path)
                triplet['candidate_color_rgb'] = candidate_color
                triplet['candidate_color_name'] = rgb_to_color_name(candidate_color)
            else:
                missing_images.append(f"{triplet['candidate']}")
                triplet['candidate_color_rgb'] = None
                triplet['candidate_color_name'] = None

            # Process target image
            if target_path:
                target_color = extract_dominant_color(target_path)
                triplet['target_color_rgb'] = target_color
                triplet['target_color_name'] = rgb_to_color_name(target_color)
            else:
                missing_images.append(f"{triplet['target']}")
                triplet['target_color_rgb'] = None
                triplet['target_color_name'] = None

            if candidate_path or target_path:
                processed_count += 1

        # Save updated JSON
        out_file = output_dir / f"cap.{dress_type}.{split}.json"
        with open(out_file, 'w') as f:
            json.dump(triplets, f, indent=2)

        print(f"✅ Saved updated triplets to {out_file}")
        print(f"📊 Processed: {processed_count}/{len(triplets)} triplets")
        if missing_images:
            print(f"⚠️  Missing {len(set(missing_images))} unique images")
            # Optionally save missing images list
            missing_file = output_dir / f"missing_images_{dress_type}_{split}.txt"
            with open(missing_file, 'w') as f:
                f.write('\n'.join(sorted(set(missing_images))))
            print(f"📝 Missing images list saved to {missing_file}")

print("🎉 Color processing complete!")

🎨 Processing /content/fashion-iq/captions/cap.dress.train.json...


100%|██████████| 5985/5985 [02:12<00:00, 45.29it/s]


✅ Saved updated triplets to /content/fashion-iq/captions/captions_with_colors2/cap.dress.train.json
📊 Processed: 5985/5985 triplets
⚠️  Missing 41 unique images
📝 Missing images list saved to /content/fashion-iq/captions/captions_with_colors2/missing_images_dress_train.txt
🎨 Processing /content/fashion-iq/captions/cap.dress.val.json...


100%|██████████| 2017/2017 [00:44<00:00, 45.83it/s]


✅ Saved updated triplets to /content/fashion-iq/captions/captions_with_colors2/cap.dress.val.json
📊 Processed: 2016/2017 triplets
⚠️  Missing 19 unique images
📝 Missing images list saved to /content/fashion-iq/captions/captions_with_colors2/missing_images_dress_val.txt
🎨 Processing /content/fashion-iq/captions/cap.dress.test.json...


  0%|          | 0/2024 [00:00<?, ?it/s]


KeyError: 'target'